In [1]:
import json
import pandas as pd
import numpy as np
import mpqa

In [2]:
df_data = pd.read_pickle('new_data.pickle')

In [4]:
df_data.columns.size

13317

In [6]:
len(df_data)

8016

In [7]:
df_data.info()

<class 'pandas.sparse.frame.SparseDataFrame'>
Int64Index: 8016 entries, 0 to 8015
Columns: 13317 entries, after_0000 to word_4433
dtypes: float64(13317)
memory usage: 418.2 KB


In [3]:
ddf = df_data.to_dense()

In [10]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8016 entries, 0 to 8015
Columns: 13317 entries, after_0000 to word_4433
dtypes: float64(13317)
memory usage: 814.5 MB


In [4]:
df_labels = pd.read_pickle('new_labels.pickle')

In [15]:
df_labels['c_pol'].value_counts()

-0.75    2606
-1.50    2504
 0.75    1811
 1.50     694
-2.00     240
-3.00      71
-2.25      35
 2.25      18
 3.00      11
-3.50      10
 2.00       9
-4.00       5
-1.25       1
-3.75       1
dtype: int64

In [5]:
X = ddf.values

In [6]:
y = df_labels['c_pol'].map(lambda k: 0 if k < 0 else 1).values

In [7]:
X.shape, y.shape

((8016, 13317), (8016,))

In [8]:
from sklearn.svm import SVC

In [9]:
svc_1 = SVC(kernel='linear')

In [10]:
from sklearn.cross_validation import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [12]:
X_train.shape, y_train.shape

((6012, 13317), (6012,))

In [17]:
from sklearn import metrics

In [13]:
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    print "Accuracy on training set:"
    print clf.score(X_train, y_train)
    print "Accuracy on testing set:"
    print clf.score(X_test, y_test)
    
    y_pred = clf.predict(X_test)
    
    print "Classification Report:"
    print metrics.classification_report(y_test, y_pred)
    print "Confusion Matrix:"
    print metrics.confusion_matrix(y_test, y_pred)

In [14]:
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem

In [15]:
def evaluate_cross_validation(clf, X, y, K):
    # create a k-fold croos validation iterator
    cv = KFold(len(y), K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score 
    # method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print scores
    print ("Mean score: {0:.3f} (+/-{1:.3f})").format(
            np.mean(scores), sem(scores))

In [16]:
evaluate_cross_validation(svc_1, X_train, y_train, 5)
train_and_evaluate(svc_1, X_train, X_test, y_train, y_test)

[ 0.85203658  0.8478803   0.85607321  0.85357737  0.84941764]
Mean score: 0.852 (+/-0.001)
Accuracy on training set:
0.966733200266
Accuracy on testing set:
0.857285429142
Classification Report:


NameError: global name 'metrics' is not defined

In [18]:
y_pred = svc_1.predict(X_test)

In [20]:
print "Classification Report:"
print metrics.classification_report(y_test, y_pred)
print "Confusion Matrix:"
print metrics.confusion_matrix(y_test, y_pred)

Classification Report:
             precision    recall  f1-score   support

          0       0.89      0.91      0.90      1364
          1       0.79      0.75      0.77       640

avg / total       0.86      0.86      0.86      2004

Confusion Matrix:
[[1239  125]
 [ 161  479]]
